In [ ]:
# запускаем в colab или локально?
try:
    from google.colab import drive
    colab = True
except ImportError:
    colab = False

print(f"colab: {colab}")

In [ ]:
# установка необходимых пакетов в colab
if colab:
    ! pip install rootutils -q
    ! pip install torchmetrics -q
    ! pip install torchinfo -q
    ! pip install livelossplot -q

In [ ]:
# монтирование google диска и установка
# рабочей директории в `computer-vision`

import os
import rootutils

root = rootutils.setup_root(".", indicator="homeworks", pythonpath=True)

if colab:
    drive.mount("/content/drive", force_remount=True)
    os.chdir("drive/MyDrive/computer-vision")
else:
    os.chdir(root)

print(f"working directory: {os.getcwd()}")

In [ ]:
# создание директории для данных

from pathlib import Path

if colab:
    DATA_DIR = Path("/content/data")
else:
    DATA_DIR = root / "data"

DATA_DIR.mkdir(exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")

In [ ]:
# настройки для matplotlib

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format="retina"

plt.style.use("seaborn-v0_8-notebook")

## **Полносвязные нейронные сети. Методы регуляризации**

### **Полносвязные нейронные сети**

**Полносвязные нейронные сети** (Fully Connected Neural Networks, FCNN).

Другое название - **многослойный перцептрон** (Multilayer Perceptron, MLP).

#### **Пример полносвязной сети**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/notebooks/figs/02/fcnn.jpg" width="600px"/>
    </figure>
</center>

- У этой сети один входной слой, один выходной слой и три скрытых слоя.

- Параметры нейронной сети 
    $$
        \mathbf{w} = \left\{
            \boldsymbol{\beta}_0,\boldsymbol{\Omega}_0,
            \boldsymbol{\beta}_1,\boldsymbol{\Omega}_1,
            \boldsymbol{\beta}_2,\boldsymbol{\Omega}_2,
            \boldsymbol{\beta}_3,\boldsymbol{\Omega}_3
        \right\}
    $$
    Число параметров равно:
    $$
        (D_1\cdot D_i + D_1) + (D_2\cdot D_1 + D_2) + (D_3\cdot D_2 + D_3) + (D_0\cdot D_3 + D_0) = 43
    $$

- Вычисления слоев нейронной сети:
    $$
        \mathbf{h}_1 = \mathbf{a}\left[\boldsymbol{\beta}_0 + \mathbf{\Omega}_0 \mathbf{x}\right]
    $$
    $$
        \mathbf{h}_2 = \mathbf{a}\left[\boldsymbol{\beta}_1 + \mathbf{\Omega}_1 \mathbf{h}_1\right]
    $$
    $$
        \mathbf{h}_3 = \mathbf{a}\left[\boldsymbol{\beta}_2 + \mathbf{\Omega}_2 \mathbf{h}_2\right]
    $$
    $$
        \mathbf{y} = \boldsymbol{\beta}_3 + \mathbf{\Omega}_3 \mathbf{h}_3
    $$

    где $\mathbf{a}$ - функция активации, обычно это $\mathrm{ReLU}$ (Rectified Linear Unit):
    <center>
        <figure>
            <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/notebooks/figs/02/relu.png" width="200px"/>
        </figure>
    </center>

    $$
        a[z] = \mathrm{ReLU}[z] =
        \begin{cases}
            0 & z < 0\\
            z & z \geqslant 0
        \end{cases}
    $$

- Сеть называется **полносвязной**, поскольку каждый элемент (node) связан со всеми предыдущими элементами.


#### **Полносвязные нейронной сети в PyTorch**

Напишем класс для нашей полносвязной нейронной сети в PyTorch:

In [ ]:
from torch import nn

class Net(nn.Module):
    def __init__(self, Di=3, D1=4, D2=2, D3=3, Do=2):
        super().__init__()

        self.lin1 = nn.Linear(Di, D1)
        self.lin2 = nn.Linear(D1, D2)
        self.lin3 = nn.Linear(D2, D3)
        self.lin4 = nn.Linear(D3, Do)

        self.act = nn.ReLU() 
         
    def forward(self, x):
        h1 = self.act(self.lin1(x))
        h2 = self.act(self.lin2(h1))
        h3 = self.act(self.lin3(h2))
        
        y = self.lin4(h3)

        return y

Протестируем работу сети:

In [ ]:
import torch

# модель сети
model = Net()

# батч размера 5 из случайных чисел
x = torch.rand((5, 3))

# forward pass
y = model(x)

print(f"x.shape: {x.shape}")
print(f"\nx: {x}")
print(f"\ny.shape: {y.shape}")
print(f"\ny: {y}")

Выведем информацию о архитектуре сети:

In [ ]:
import torchinfo

print(torchinfo.summary(model, input_size=x.shape, device="cpu"))

Можно написать более компактную реализацию с помощью `torch.nn.Sequential`:

In [ ]:
class Net(nn.Module):
    def __init__(self, Di=3, D1=4, D2=2, D3=3, Do=2):
        super().__init__()

        act = nn.ReLU()

        self.layers = nn.Sequential(
            nn.Linear(Di, D1),
            act,
            nn.Linear(D1, D2),
            act,
            nn.Linear(D2, D3),
            act,
            nn.Linear(D3, Do),
        )
         
    def forward(self, x):
        y = self.layers(x)

        return y
    
    model = Net()

    print(torchinfo.summary(model, input_size=x.shape, device="cpu"))

### **Классификация изображений Fashion-MNIST**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/at/lesson2/notebooks/figs/02/fashion_mnist.jpg" width="600px"/>
    </figure>
</center>
Обучим полносвязную нейронную сеть классифицировать изображения из датасета Fashion-MNIST:

- Размер изображений: 28x28 пикселей

- Цветовое пространство: grayscale

- Количество классов: 10 (одежда)

- Размер набора данных: 60 000 обучающих и 10 000 тестовых изображений


#### **Dataset and DataLoader**

В torchvision есть класс `torchvision.datasets.FashionMNIST` для работы с этим датасетом:

In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms as T

import numpy as np

# 1. Define transformations: Convert images to PyTorch tensors
transform = T.Compose([
    T.ToTensor()
])
             
# 2. Load the Fashion-MNIST datasets (downloads if not present)
train_dataset = FashionMNIST(
    root=DATA_DIR, 
    train=True, 
    download=True, 
    transform=transform
)

test_dataset = FashionMNIST(
    root=DATA_DIR, 
    train=False, 
    download=True, 
    transform=transform
)

# 3. Define the class names
class_names = {
    0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat',
    5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'
}

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Посмотрим на один из сэмплов:

In [ ]:
# x - tensor
# y - int
x, y = train_dataset[2026]

class_name = class_names[y]

print(f"x.shape: {x.shape}")
print(f"label: {y}")
print(f"class name: {class_name}")

In [ ]:
import matplotlib.pyplot as plt

# де-нормализация изображения
image = (255 * x.squeeze()).numpy().astype(np.uint8)

# построение изображения
plt.figure(figsize=(3, 3))
plt.imshow(image, cmap="gray")
plt.title(f"class name: {class_name}")
plt.axis("off");

Мы напишем собственную реализацию `torch.utils.data.Dataset` для Fashion-MNIST.

Загрузим и распакуем изображения с google диска:

In [ ]:
from src.utils import download_and_extract

download_and_extract("1KpBql_Rpc1dtBSwFTT7UZdNLUoDde6YQ", "fashion-mnist.zip", DATA_DIR)

Напишем класс для итерации по данным по одному сэмплу (x, y):

- x - изображение 28 x 28

- y - метка класса 0, 1, 2, ..., 9

In [ ]:
import cv2
import numpy as np
from pathlib import Path

from torch.utils.data import Dataset

class DataSet(Dataset):
    def __init__(self, data_dir):
        super().__init__()

        self.files = list(Path(data_dir).glob("**/*.png"))
        self.labels = [int(f.parent.name) for f in self.files]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        image = cv2.imread(self.files[index], cv2.IMREAD_GRAYSCALE)

        # нормализация изображения в диапазон [0, 1]
        x = image.astype(np.float32) / 255

        # преобразование размера
        # 28 x 28 в 1 x 28 x 28
        x = x[None]

        # метка класса 
        y = self.labels[index]

        return x, y

Создадим датасеты для обучения и тестирования:

In [ ]:
dataset = DataSet(DATA_DIR / "fashion-mnist/train")
test_dataset = DataSet(DATA_DIR / "fashion-mnist/test")

# число изображений в датасетах
print(f"dataset size: {len(dataset)}")
print(f"test_dataset size: {len(test_dataset)}")

Посмотрим на один из сэмплов:

In [ ]:
# x - numpy массив
# y - int
x, y = dataset[2026]

# имя класса
class_name = class_names[y]

print(f"x shape: {x.shape}")
print(f"y: {y}")

# де-нормализация изображения
image = (255 * x[0]).astype(np.uint8)

# построение изображения
plt.figure(figsize=(3, 3))
plt.imshow(image, cmap="gray")
plt.title(f"class: {class_name}")
plt.axis("off");

Разделим `dataset` на датасеты для обучения и валидации:

In [ ]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(
    dataset,
    [50_000, 10_000]
)

print(f"train_dataset size: {len(train_dataset)}")
print(f"val_dataset size: {len(val_dataset)}")

Создадим загрузчики батчей для обучения и валидации:

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,   # датасет
    batch_size=1024, # размер батча
    num_workers=0,   # число процессов для формирования батчей
    shuffle=True,    # перемешивать датасет в конце каждой эпохи
    drop_last=True   # выбрасывать последний неполный батч в эпохе
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1024,
    num_workers=0,
    shuffle=False,
    drop_last=False
)

# число батчей
print(f"train_loader size: {len(train_loader)}")
print(f"val_loader size: {len(val_loader)}")

Сгенерируем один батч из данных для обучения: 

In [ ]:
# для изображений размер батча должен
# иметь вид B x C x H x W
# (размер батча) x (число цветовых каналов) x (высота изображения) x (ширина изображения)

# выборка случайного батча данных из датасета
# x, y - это тензоры
x, y = next(iter(train_loader))

# приведение к нужному типу данных
x = x.float()
y = y.long()

print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}\n")

print(f"x.min(): {x.min()}")
print(f"x.max(): {x.max()}")

#### **Модель классификатора**

Создадим полносвязную нейронную сеть для классификации с одним скрытым слоем:

In [ ]:
class Classifier(nn.Module):
    def __init__(self, D):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(28 * 28, D),
            nn.ReLU(),
            nn.Linear(D, 10),
        )

    def forward(self, x):
        # преобразуем размер 
        # B x 1 x 28 x 28 -> B x (28 * 28)
        x = x.view(-1, 28 * 28)
        
        y = self.layers(x)

        return y

Протестируем модель сети:

In [ ]:
model = Classifier(D=196)

x, y = next(iter(val_loader))

y_hat = model(x)

print(f"x shape: {x.shape}")
print(f"y_hat shape: {y_hat.shape}")

Выведем архитектуру модели:

In [ ]:
print(torchinfo.summary(model, input_size=x.shape, device="cpu"))

#### **Функция потерь и метрика**

Функция потерь для задачи мультиклассовой классификации - средняя кросс-энтропия на батче (Cross Entropy):
$$
    \mathbf{CE} = 
    -\frac{1}{|\mathcal{I}_t|}
    \sum_{i\in\mathcal{I}_t}\mathbf{p}_i\cdot\ln \widehat{\mathbf{p}}_i
$$
где $\mathbf{p}_i$ - one-hot вектор истинных вероятностей изображения с номером $i$:
$$
   \mathbf{p}_i = (0\,,0\,,\ldots\,,1\,,0\,,\ldots\,,0)
$$ 
$\widehat{\mathbf{p}}_i$ - вектор предсказанных моделью вероятностей изображения с номером $i$:
$$
    \widehat{\mathbf{p}}_i = \mathrm{softmax}[\widehat{\mathbf{y}}_i] = 
    \frac{\exp[\widehat{\mathbf{y}}_i]}{\sum\limits_j \exp[\widehat{y}_{ij}]}
$$

Загрузчик данных возвращает $\left\{\mathbf{y}_i\right\}_{i\in\mathcal{I}_t}$.

Модель возвращает $\left\{\widehat{\mathbf{y}}_i\right\}_{i\in\mathcal{I}_t}$.

Все необходимые преобразования в вероятности происходят в классе `torch.nn.CrossEntropyLoss()`:

In [ ]:
loss_fn = nn.CrossEntropyLoss()

x, y = next(iter(train_loader))

y_hat = model(x.float())

# вычисление loss функции по предсказанию `y_hat` и реальному `y`
# вычисления softmax и one-hot encoding `y` реализовано в `loss_fn` 
loss = loss_fn(y_hat, y)

print(f"loss value = {loss}")

В качестве оптимизатора возьмем SGD:

In [ ]:
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

В качестве метрики качества возьмем среднюю точность по классам:

In [ ]:
from torchmetrics.classification import MulticlassAccuracy

# вычисляет точность предсказания по каждому классу
metric = MulticlassAccuracy(num_classes=10, average=None)

Протестируем метрику качества:

In [ ]:
# итератор по данным
iterator = iter(val_loader)

# получим первый батч 
x, y = next(iterator)
y_hat = model(x.float())

# добавим в метрику
metric.update(y_hat, y.long())

# получим следующий батч
x, y = next(iterator)
y_hat = model(x.float())

# добавим в метрику
metric.update(y_hat, y.long())

# вычисление точности по каждому классу
accuracy = metric.compute()

# средняя точность по классам
mean_accuracy = accuracy.mean()

print(f"class accuracy: {accuracy}")
print(f"mean accuracy: {mean_accuracy}")

#### **Обучение модели**

Соберем все вместе и напишем полный цикл обучения и валидации модели по эпохам:

In [ ]:
from dataclasses import dataclass

from torchmetrics.aggregation import MeanMetric
from livelossplot import PlotLosses

from src.utils import set_seed

@dataclass
class Config:
    seed = 0xC0FFEE
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_workers = 0
    # гиперпараметры
    epochs = 10
    learning_rate = 0.1
    batch_size = 1000
    D = 196
    
cfg = Config()

# для воспроизводимости результатов обучения
set_seed(cfg.seed)

# загрузчик данных для обучения
train_loader = DataLoader(
    train_dataset,
    batch_size=cfg.batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=cfg.num_workers
)

# загрузчик данных для валидации
val_loader = DataLoader(
    val_dataset,
    batch_size=cfg.batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=cfg.num_workers
)

# модель классификатора
model = Classifier(D=cfg.D)
model = model.to(cfg.device)

# оптимизатор 
optimizer = torch.optim.SGD(params=model.parameters(), lr=cfg.learning_rate)

# функция потерь
loss_fn = nn.CrossEntropyLoss()

# для построения графиков во время обучения
plot = PlotLosses(figsize=(12, 3))

# лучшее значение метрики качества
best_metric = 0

for epoch in range(cfg.epochs):
    # ЭПОХА ОБУЧЕНИЯ МОДЕЛИ

    # модель переведем в режим обучения
    model.train()

    # значения loss функции на батчах одной эпохи
    batch_losses = torch.empty(0)

    metric = MulticlassAccuracy(num_classes=10, average=None)

    # для вычислений среднего значения метрики
    mean_metric = MeanMetric()

    # выбираем случайные батчи из обучающих данных
    for x, y in train_loader:
        x = x.float().to(cfg.device)
        y = y.long().to(cfg.device)

        # предсказание модели
        y_hat = model(x)

        # вычисление loss функции по предсказанию `y_hat` и реальному `y`
        loss = loss_fn(y_hat, y)

        # обнулим все производные перед их вычислениями
        # чтобы не было gradient accumulation
        optimizer.zero_grad()

        # вычисление производных loss функция по параметрам
        # c помощью метода обратного распространения ошибки (backpropagation)
        loss.backward()

        # обновление параметров модели
        optimizer.step()

        mean_metric.update(loss.detach().cpu())
        metric.update(y_hat.detach().cpu(), y.cpu())

    # среднее значение функции потерь в эпохе обучения
    train_epoch_loss = mean_metric.compute()
    
    # среднее значение метрики в эпохе обучения
    train_epoch_metric = metric.compute().mean()
    
    # ЭПОХА ВАЛИДАЦИИ МОДЕЛИ

    # модель переведем в режим валидации
    model.eval()

    batch_losses = torch.empty(0)

    metric = MulticlassAccuracy(num_classes=10, average=None)
    mean_metric = MeanMetric()

    for x, y in val_loader:
        x = x.float().to(cfg.device)
        y = y.long().to(cfg.device)

        # этот контекст рекомендуют использовать для ускорения вычислений при валидации
        with torch.no_grad():
            y_hat = model(x)
        
        loss = loss_fn(y_hat, y)

        mean_metric.update(loss.cpu())
        metric.update(y_hat.cpu(), y.cpu())

    # среднее значение функции потерь в эпохе валидации
    val_epoch_loss = mean_metric.compute()

    # среднее значение метрики в эпохе валидации
    val_epoch_metric = metric.compute().mean()

    # сохраняем модель, если на валидации метрика улучшилась
    if best_metric < val_epoch_metric:
        best_metric = val_epoch_metric
        torch.save(model.state_dict(), DATA_DIR / "classifier.pth")

    # построение графиков во время обучения
    plot.update({
        "loss": train_epoch_loss,
        "val_loss": val_epoch_loss,
        "accuracy": train_epoch_metric,
        "val_accuracy": val_epoch_metric
    })

    plot.send()

#### **Тестирование модели**

Загрузим веса обученной модели для тестирования:

In [ ]:
state_dict = torch.load(DATA_DIR / "classifier.pth", weights_only=True)

model = Classifier(cfg.D)
model.load_state_dict(state_dict, strict=True)

Протестируем на CPU модель на тестовых данных:

In [ ]:
test_loader = DataLoader(
    test_dataset,
    batch_size=1000,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

# модель в режим тестирования
model.eval()

metric = MulticlassAccuracy(num_classes=10, average=None)

# перебираем все батчи из тестовых данных
for x, y in test_loader:
    x = x.float()
    y = y.long()

    # при тестировании, для скорости
    # рекомендуют использовать `torch.inference_mode()` контекст
    with torch.inference_mode():
        y_hat = model(x)

    metric.update(y_hat, y)

class_accuracy = metric.compute()
mean_accuracy = class_accuracy.mean()

for label, accuracy in enumerate(class_accuracy):
    name = class_names[label]
    print(f"{name:>15}: {100 * accuracy.item():.2f} %")

print(f"\nmean_accuracy: {100 * mean_accuracy.item():.2f} %")



Возьмем одно тестовое изображение:

In [ ]:
x, y = test_dataset[102]

image = (255 * x[0]).astype(np.uint8)
name = class_names[y]

plt.figure(figsize=(3, 3))
plt.imshow(image, cmap="gray")
plt.title(f"Class: {name}")
plt.axis("off");

и сделаем на нем предсказание модели:

In [ ]:
# преобразование из 28 x 28 numpy array
# в 1 x 1 x 28 x 28 pytorch tensor
x = torch.from_numpy(image)
x = x.unsqueeze(0).unsqueeze(1)
x = x.float() / 255

with torch.inference_mode():
    y_hat = model(x)

# предсказанная метка класса - 
# это индекс максимального элемента выхода модели
pred_label = torch.argmax(y_hat, dim=1)

# преобразование из тензора в число
pred_label = pred_label.item()

# имя предсказанного класса
pred_class = class_names[pred_label]

print(f"Predicted class: {pred_class}")